In [2]:
import networkx as nx
import time
import numpy as np
from collections import defaultdict
from collections import deque

In [3]:
# update_coreness.py의 함수들을 여기에 포함
def mcdegree_node(G, u, K):
    k = K[u]
    return sum(1 for nb in G.neighbors(u) if K[nb] >= k)

def mcdegree_dict(G, nodes, K=None):
    mcd = {}
    for u in nodes:
        mcd[u] = mcdegree_node(G, u, K)
    return mcd

def propagate_dismissal(G, K, MCD, cd, dismissed, visited, k, v):
    dismissed[v] = True
    K[v] -= 1
    for w in G.neighbors(v):
        if K[w] == k:
            if not visited[w]:
                if w not in MCD:
                    MCD[w] = mcdegree_node(G, w, K)
                    MCD[w] += 1
                cd[w] += MCD[w]
                visited[w] = True
            cd[w] -= 1
            if not dismissed[w] and cd[w] < k:
                propagate_dismissal(G, K, MCD, cd, dismissed, visited, k, w)

def prepareRCDs(G, K, MCD, u, v):
    node_set = set()
    for node in (u, v):
        if node in G:
            node_set.add(node)
            node_set.update(G.neighbors(node))
    
    for node in node_set:
        MCD[node] = mcdegree_node(G, node, K)

def recomputeRCD(G, K, MCD, dismissed):
    node_set = set()
    for node in dismissed:
        if dismissed[node]:
            if node not in node_set:
                MCD[node] = mcdegree_node(G, node, K)
                node_set.add(node)
            
            for nb in G.neighbors(node):
                if nb not in node_set:
                    MCD[nb] = mcdegree_node(G, nb, K)
                    node_set.add(nb)

def traversal(G, K, MCD, u, v):
    r = u
    if K[v] < K[u]: 
        r = v
    
    prepareRCDs(G, K, MCD, u, v)
    
    visited = defaultdict(lambda: False)
    dismissed = defaultdict(lambda: False)
    cd = defaultdict(lambda: 0)
    k = K[r]
    
    if K[u] != K[v]:
        if r in G.nodes():
            visited[r] = True
            if r not in MCD: 
                MCD[r] = mcdegree_node(G, r, K)
            cd[r] = MCD[r]
            if cd[r] < k:
                propagate_dismissal(G, K, MCD, cd, dismissed, visited, k, r)
    else:
        if u in G.nodes():
            visited[u] = True
            if u not in MCD: 
                MCD[u] = mcdegree_node(G, u, K)
            cd[u] = MCD[u]
            if cd[u] < k:
                propagate_dismissal(G, K, MCD, cd, dismissed, visited, k, u)
        if v in G.nodes():
            visited[v] = True
            if v not in MCD: 
                MCD[v] = mcdegree_node(G, v, K)
            cd[v] = MCD[v]
            if not dismissed[v] and cd[v] < k:
                propagate_dismissal(G, K, MCD, cd, dismissed, visited, k, v)
    recomputeRCD(G, K, MCD, dismissed)

def run_update_coreness(G, K_, MCD_, node, f='F'):
    K = K_.copy()
    MCD = MCD_.copy()
    edge_list = set(G.edges(node))
    for u, v in edge_list:
        G.remove_edge(u, v)
        traversal(G, K, MCD, u, v)
    del K[node]
    if f == 'T':
        G.add_edges_from(edge_list)
    else:
        G.remove_node(node)
    return K, MCD


In [5]:
def load_graph(filename):
    """그래프 로드 함수"""
    G = nx.read_edgelist(filename, nodetype=int)
    G.remove_edges_from(nx.selfloop_edges(G))
    mapping = {node: idx for idx, node in enumerate(G.nodes())}
    G = nx.relabel_nodes(G, mapping)
    return G

def compare_coreness_methods():
    """nx.core_number와 update_coreness 방법 비교"""
    
    print("=== Coreness 계산 방법 비교 테스트 ===\n")
    
    # 테스트 데이터셋들 (실제 경로에 맞게 수정 필요)
    datasets = [
        ("Karate", "./dataset/karate/network.dat"),
        ("Football", "./dataset/football/network.dat"),
        ("Mexican", "./dataset/mexican/network.dat"),
        ("Strike", "./dataset/strike/network.dat")
    ]
    
    results = []
    
    for name, path in datasets:
        print(f"--- {name} 데이터셋 테스트 ---")
        
        try:
            # 그래프 로드
            G = load_graph(path)
            print(f"노드 수: {G.number_of_nodes()}, 엣지 수: {G.number_of_edges()}")
            
            # 테스트할 노드들 선택
            test_nodes = list(G.nodes())[:min(3, len(G.nodes()))]  # 최대 3개 노드 테스트
            print(f"테스트 노드: {test_nodes}")
            
            accuracy_results = []
            
            for test_node in test_nodes:
                print(f"\n노드 {test_node} 제거 테스트:")
                
                # 1. nx.core_number 방식 (노드 제거 후 새로 계산)
                G_nx = G.copy()
                start_time = time.time()
                G_nx.remove_node(test_node)
                nx_result = nx.core_number(G_nx)
                nx_time = time.time() - start_time
                
                # 2. update_coreness 방식 (점진적 업데이트)
                G_update = G.copy()
                initial_coreness = nx.core_number(G_update)
                initial_MCD = mcdegree_dict(G_update, G_update.nodes(), initial_coreness)
                
                start_time = time.time()
                update_result, _ = run_update_coreness(G_update, initial_coreness, initial_MCD, test_node, 'F')
                update_time = time.time() - start_time
                
                # 3. 결과 비교 (공통 노드들만)
                common_nodes = set(nx_result.keys()) & set(update_result.keys())
                is_accurate = True
                differences = []
                
                for node in common_nodes:
                    if nx_result[node] != update_result[node]:
                        is_accurate = False
                        differences.append((node, nx_result[node], update_result[node]))
                
                accuracy_results.append(is_accurate)
                
                print(f"  nx.core_number 시간: {nx_time:.4f}s")
                print(f"  update_coreness 시간: {update_time:.4f}s")
                print(f"  속도 향상: {nx_time / update_time:.2f}x" if update_time > 0 else "무한대")
                print(f"  정확성 일치: {is_accurate}")
                
                if not is_accurate:
                    print(f"  차이점 (노드, nx값, update값): {differences[:5]}")  # 최대 5개만 출력
                
                # 4. 샘플 결과 출력 (처음 5개 노드)
                sample_nodes = list(common_nodes)[:5]
                print(f"  샘플 결과 비교:")
                for node in sample_nodes:
                    print(f"    노드 {node}: nx={nx_result[node]}, update={update_result[node]}")
            
            # 전체 정확성
            overall_accuracy = all(accuracy_results)
            
            result = {
                'dataset': name,
                'nodes': G.number_of_nodes(),
                'edges': G.number_of_edges(),
                'overall_accuracy': overall_accuracy,
                'accuracy_rate': sum(accuracy_results) / len(accuracy_results) if accuracy_results else 0,
                'test_count': len(test_nodes)
            }
            results.append(result)
            
            print(f"\n전체 정확성: {overall_accuracy} ({sum(accuracy_results)}/{len(accuracy_results)})")
            print("-" * 50)
            
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")
            print("경로를 확인해주세요.\n")
            continue
        except Exception as e:
            print(f"오류 발생: {e}\n")
            continue
    
    # 결과 요약
    if results:
        print("\n=== 전체 결과 요약 ===")
        total_accuracy = sum(r['accuracy_rate'] for r in results) / len(results)
        print(f"평균 정확성: {total_accuracy:.2%}")
        print(f"완전 정확한 데이터셋: {sum(1 for r in results if r['overall_accuracy'])}/{len(results)}")
    
    return results

# 테스트 실행
if __name__ == "__main__":
    results = compare_coreness_methods()

=== Coreness 계산 방법 비교 테스트 ===

--- Karate 데이터셋 테스트 ---
노드 수: 34, 엣지 수: 78
테스트 노드: [0, 1, 2]

노드 0 제거 테스트:
  nx.core_number 시간: 0.0003s
  update_coreness 시간: 0.0010s
  속도 향상: 0.33x
  정확성 일치: True
  샘플 결과 비교:
    노드 1: nx=3, update=3
    노드 2: nx=3, update=3
    노드 3: nx=3, update=3
    노드 4: nx=2, update=2
    노드 5: nx=2, update=2

노드 1 제거 테스트:
  nx.core_number 시간: 0.0003s
  update_coreness 시간: 0.0013s
  속도 향상: 0.23x
  정확성 일치: True
  샘플 결과 비교:
    노드 0: nx=3, update=3
    노드 2: nx=3, update=3
    노드 3: nx=3, update=3
    노드 4: nx=3, update=3
    노드 5: nx=3, update=3

노드 2 제거 테스트:
  nx.core_number 시간: 0.0004s
  update_coreness 시간: 0.0007s
  속도 향상: 0.52x
  정확성 일치: True
  샘플 결과 비교:
    노드 0: nx=3, update=3
    노드 1: nx=3, update=3
    노드 3: nx=3, update=3
    노드 4: nx=3, update=3
    노드 5: nx=3, update=3

전체 정확성: True (3/3)
--------------------------------------------------
--- Football 데이터셋 테스트 ---
노드 수: 115, 엣지 수: 613
테스트 노드: [0, 1, 2]

노드 0 제거 테스트:
  nx.core_number 시간: 0.0010s
  update_